In [1]:
import pandas as pd
from selenium import webdriver

In [2]:
options = webdriver.EdgeOptions()
options.add_experimental_option(
    "prefs", {
        # block image loading
        "profile.managed_default_content_settings.images": 2,
    }
)

browser = webdriver.Edge(options=options)

browser.get('https://www.imdb.com/search/title/?title_type=feature,tv_movie&user_rating=6,10&sort=user_rating,desc&num_votes=50000,')

pagination_btn = browser.find_element("xpath","//div[contains(@class,'pagination-container')]//button")
for _ in range(300):
    # browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        browser.execute_script("arguments[0].scrollIntoView();", pagination_btn)
        pagination_btn.click()
        browser.execute_script("arguments[0].scrollIntoView();", pagination_btn)
    except:
        browser.implicitly_wait(10)
        

def get_data(xpath, element=browser):
    try:
        return element.find_element("xpath",xpath).text
    except:
        return None

elements = browser.find_elements("xpath","//ul[contains(@class, 'ipc-metadata-list')]/li")
print(len(elements))
movies = []

for element in elements:
    try:
        movies.append({
            "link": element.find_element("xpath",".//a[contains(@class, 'ipc-title-link-wrapper')]").get_attribute("href"),
            "description": get_data(".//div[@class='ipc-html-content-inner-div']",element),
            "rating":get_data(".//span[contains(@class,'ipc-rating-star--imdb')]",element).split(" ")[0].strip(),
            "votes": get_data(".//div[contains(@class,'dli-plot-container')]/following-sibling::div",element).replace("Votes",""),
            "year": get_data(".//div[contains(@class,'dli-title-metadata')]/span[1]",element),
            "duration": get_data(".//div[contains(@class,'dli-title-metadata')]/span[2]",element),
            "certificate": get_data(".//div[contains(@class,'dli-title-metadata')]/span[3]",element),
            "meta_score": get_data(".//span[contains(@class,'metacritic-score-box')]",element),
        })
        print("Movies count:",len(movies))
    except:
        pass

for movie in movies:
    try:
        browser.get(movie["link"])
        expand_btn = browser.find_element("xpath","//button[@data-testid='title-pc-expand-more-button']")
        while True:
            try:
                expand_btn.click()
                break
            except:
                pass  
        movie["title"] = get_data("//h1")
        print(movie["title"])
        movie["type"] = 'Movie'
        movie["popularity"] = get_data("//div[1]/div[1]/div[3]/a[1]/span[1]/div[1]/div[2]/div[1]")
        movie["genre"] = ",".join([genre.text for genre in browser.find_elements("xpath","//div[@data-testid='genres']//a")])
        movie["director"] = ",".join([director.text for director in browser.find_elements("xpath","//div[1]/div[1]/ul[1]/li[1]/div[1]//li/a")])
        movie["writer"] = ",".join([writer.text for writer in browser.find_elements("xpath","//div[1]/div[1]/ul[1]/li[2]/div[1]//li/a")])
        movie["stars"] = ",".join([star.text for star in browser.find_elements("xpath","//div[1]/div[1]/ul[1]/li[3]/div[1]//li/a")])
        movie["country"] = ",".join([country.text for country in browser.find_elements("xpath","//li[@data-testid='title-details-origin']/div/ul/li")])
        movie["language"] = ",".join([language.text for language in browser.find_elements("xpath","//li[@data-testid='title-details-languages']/div/ul/li")])
        movie["production_company"] = ",".join([production_company.text for production_company in browser.find_elements("xpath","//li[@data-testid='title-details-companies']/div/ul/li")])
        movie["worldwide_gross"] = get_data("//li[@data-testid='title-boxoffice-cumulativeworldwidegross']/div//span")
    except:
        pass

browser.close()

df = pd.DataFrame(movies)
df.to_csv("imdb.csv",index=False)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=121.0.2277.83)
Stacktrace:
	GetHandleVerifier [0x00007FF7C83CAD02+60770]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF7C83584B2+228274]
	(No symbol) [0x00007FF7C816ADBB]
	(No symbol) [0x00007FF7C81422BD]
	(No symbol) [0x00007FF7C81E3091]
	(No symbol) [0x00007FF7C81F8BC3]
	(No symbol) [0x00007FF7C81DC143]
	(No symbol) [0x00007FF7C81AD4A8]
	(No symbol) [0x00007FF7C81AC317]
	(No symbol) [0x00007FF7C81AD041]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF7C857B47C+1039884]
	(No symbol) [0x00007FF7C8230D65]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF7C82CA82C+36956]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF7C82C2352+2946]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF7C8579EC3+1034323]
	Microsoft::Applications::

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   link                150 non-null    object
 1   rating              150 non-null    object
 2   votes               150 non-null    object
 3   year                150 non-null    object
 4   duration            150 non-null    object
 5   certificate         141 non-null    object
 6   meta_score          141 non-null    object
 7   title               149 non-null    object
 8   type                149 non-null    object
 9   popularity          149 non-null    object
 10  genre               149 non-null    object
 11  director            149 non-null    object
 12  writer              149 non-null    object
 13  stars               149 non-null    object
 14  country             149 non-null    object
 15  language            149 non-null    object
 16  production_company  149 no

In [4]:
print(round(3511/2))

1756
